In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from library_hw4.features.load import *
from library_hw4.features.preprocessor import *
from library_hw4.features.model import *
from abc import ABC, abstractmethod



In [2]:
#Create a class with a primary method that loads the data and returns two dataframes, one for
#train and another for test. Internally, the class can use the function defined in hw5.
#hw4/library_hw4/sample_diabetes_mellitus_data - sample_diabetes_mellitus_data.csv
data_file_path = '../sample_diabetes_mellitus_data.csv'

# Create an instance of DataLoader
data_loader = split_data(file_path=data_file_path,test_size=0.2,random_state=42)

# Load the data
data_loader.load_data()

# Get the train and test dataframes
train_df = data_loader.get_train_data()
test_df = data_loader.get_test_data()

print(train_df)
print(test_df)

      Unnamed: 0  encounter_id  hospital_id   age        bmi  \
9254        9254        185898          137  88.0  28.713767   
1561        1561        179718           83  69.0        NaN   
1670        1670        204365          118  61.0        NaN   
6087        6087        167361          118  36.0  33.861606   
6669        6669        213013           83  49.0  22.693954   
...          ...           ...          ...   ...        ...   
5734        5734        232888          118  86.0        NaN   
5191        5191        210825          118  68.0  32.549816   
5390        5390        240686          151  54.0  24.509470   
860          860        231954           33  87.0  28.566361   
7270        7270        185737          118  60.0  19.400691   

      elective_surgery         ethnicity gender  height hospital_admit_source  \
9254                 0         Caucasian      F  157.48  Emergency Department   
1561                 0         Caucasian      F  152.40              

In [3]:
#Create a preprocessor class that removes those rows that contain NaN values in the columns: age,
#gender, ethnicity.
#Create a preprocessor class that fills NaN with the mean value of the column in the columns:
#height, weight.

# Specify columns to check for NaN values
columns = ['age', 'gender', 'ethnicity']
#Create a preprocessor class that fills NaN with the mean value 
columns_fill = ['height', 'weight']

# Create an instance of RemoveNaN
dataprep = preprocessor(columns,columns_fill)

# Remove rows with NaN values in specified columns
cleaned_train_data = dataprep.remove_nan_rows(train_df)
cleaned_test_data = dataprep.remove_nan_rows(test_df)
print(cleaned_train_data)
print(cleaned_test_data)

# Fill NaN values with mean values
filled_train_data = dataprep.fill_nan_with_mean(cleaned_train_data)
filled_test_data = dataprep.fill_nan_with_mean(cleaned_test_data)

print("\nFilled Data:")
print(filled_train_data)
print(filled_test_data )

filled_test_data.isna().sum()
filled_train_data.isna().sum()

      Unnamed: 0  encounter_id  hospital_id   age        bmi  \
9254        9254        185898          137  88.0  28.713767   
1561        1561        179718           83  69.0        NaN   
1670        1670        204365          118  61.0        NaN   
6087        6087        167361          118  36.0  33.861606   
6669        6669        213013           83  49.0  22.693954   
...          ...           ...          ...   ...        ...   
5734        5734        232888          118  86.0        NaN   
5191        5191        210825          118  68.0  32.549816   
5390        5390        240686          151  54.0  24.509470   
860          860        231954           33  87.0  28.566361   
7270        7270        185737          118  60.0  19.400691   

      elective_surgery         ethnicity gender  height hospital_admit_source  \
9254                 0         Caucasian      F  157.48  Emergency Department   
1561                 0         Caucasian      F  152.40              

Unnamed: 0                        0
encounter_id                      0
hospital_id                       0
age                               0
bmi                            1275
elective_surgery                  0
ethnicity                         0
gender                            0
height                            0
hospital_admit_source           693
icu_admit_source                 17
icu_id                            0
icu_stay_type                     0
icu_type                          0
pre_icu_los_days                  0
readmission_status                0
weight                            0
albumin_apache                 4834
apache_2_diagnosis              143
apache_3j_diagnosis              40
apache_post_operative             0
arf_apache                        0
bilirubin_apache               4941
bun_apache                     1895
creatinine_apache              1787
fio2_apache                    5264
gcs_eyes_apache                  53
gcs_motor_apache            

In [4]:
#Create at least two feature classes that transform some of the columns in the data set. These
#feature classes need to have the same structure defined by an abstract parent class (Remember:
#polymorphism).


# Create instances of feature transformers
gender = {'M': 1, 'F': 0}
gender_transformer = GenderTransformer(gender)

ethnicity = {'Caucasian': 1, 'African American': 2, 'Asian': 3, 'Native American': 4, 'Hispanic':5,}
ethnicity_transformer = EthnicityTransformer(ethnicity)

# Apply the transformations
data_transformer_train= gender_transformer.transform(filled_train_data)
data_transformer_train = ethnicity_transformer.transform(filled_train_data)

print(data_transformer_train)

data_transformer_test= gender_transformer.transform(filled_test_data)
data_transformer_test = ethnicity_transformer.transform(filled_test_data)

print(data_transformer_test)


      Unnamed: 0  encounter_id  hospital_id   age        bmi  \
9254        9254        185898          137  88.0  28.713767   
1561        1561        179718           83  69.0        NaN   
1670        1670        204365          118  61.0        NaN   
6087        6087        167361          118  36.0  33.861606   
6669        6669        213013           83  49.0  22.693954   
...          ...           ...          ...   ...        ...   
5734        5734        232888          118  86.0        NaN   
5191        5191        210825          118  68.0  32.549816   
5390        5390        240686          151  54.0  24.509470   
860          860        231954           33  87.0  28.566361   
7270        7270        185737          118  60.0  19.400691   

      elective_surgery  ethnicity  gender  height hospital_admit_source  ...  \
9254                 0        1.0       0  157.48  Emergency Department  ...   
1561                 0        1.0       0  152.40                 Floor

In [6]:
model = Model(['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis'], ['diabetes_mellitus'])
model.train(train_data=filled_test_data)
filled_test_data['predictions'] = model.predict(filled_test_data)
print(model.get_accuracy(filled_test_data['diabetes_mellitus'],filled_test_data['predictions']))
print(filled_test_data.head(5))


initializing class


c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9997431810862365
      Unnamed: 0  encounter_id  hospital_id   age        bmi  \
6252        6252        275141           81  66.0  34.852863   
1731        1731        183562          118  83.0  28.215375   
4742        4742        244929          118  64.0  21.575502   
4521        4521        193781          118  21.0  34.560280   
6340        6340        178229          118  66.0  40.374334   

      elective_surgery  ethnicity  gender  height hospital_admit_source  ...  \
6252                 0        1.0       1   180.3  Emergency Department  ...   
1731                 0        1.0       0   154.9        Other Hospital  ...   
4742                 1        1.0       0   170.2  Emergency Department  ...   
4521                 1        2.0       0   162.0        Operating Room  ...   
6340                 0        3.0       0   137.2                   NaN  ...   

     wbc_apache  aids cirrhosis hepatic_failure  immunosuppression  leukemia  \
6252        7.6     0         0    